# Formation 2022  

##  Initiation à l’analyse de données Minion pour l'analyse de métagénome viraux

Created by J. Orjuela (DIADE-IRD), D. Filloux (PHIM-CIRAD) and A. Comte (PHIM-IRD) and from a formation by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) given in Septembre 2021

Septembre 2022


# TP2 : DATA CLEANING

To save time and ressources, it's better to clean your data before launching heavy analysis.

In our case, we are interrested in the viruses reads in the dataset only. Then, to alleviate the dataset, we can:
- remove the host reads (pineapple reads)
- remove the fungi reads
- remove the bacteria reads
- remove the human reads
- ...

To do so, we are going to map the reads on a reference genome or a given bank of genomes and remove the reads mapped on these genomes of the dataset.

## 1. Remove the host reads

### 1.1 downloading pineaple reference genome

For cleaning the dataset, you need to download the reference genome of the host (pineapple).

These data comme from [NCBI.](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCF_001540865.1/)

In [ ]:
cd ~/SG-ONT-2022/DATA

# download reference genome of the pineapple
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/GCA_001540865.1_Ananas_comosus_cultivar_F153.fasta

### 1.2 Mapping the reads on the reference genome

Minimap2 is a fast sequence mapping and alignment program that can find overlaps between long noisy reads, or map long reads or their assemblies to a reference genome optionally with detailed alignment

In [ ]:
minimap2 --help

In [ ]:
# create output directory
mkdir cd ~/SG-ONT-2022/CLEANING
cd ~/SG-ONT-2022/CLEANING

# Mapping
minimap2 -ax map-ont ../DATA/GCA_001540865.1_Ananas_comosus_cultivar_F153.fasta ../data/data.fastq > reads_vs_ananas.sam

Observe the quantity of mapped reads with samtools:

In [ ]:
samtools flagstats reads_vs_ananas.sam

What is the percentage of reads mapping on the pineapple genome in this dataset?

### 1.3 Separate reads from pineapple and reads not from pineapple

In [ ]:
- explication sam et flag